## Putting it all together: FIR Instance

### Intro to FIR

Now that you've learned the basics of Chisel, let's build a simple FIR (finite impulse response) filter module. FIR filters are very common in digital signal processing applications.

The basic architecture of a FIR filter is as follows:

TODO: make image

Basically, this does a elementwise multiplication of the element of the filter coefficients with the elements of the input signal and outputs the sum (also called a _convolution_).

Or, a signals definition:

$y[n] = b_0 x[n] + b_1 x[n-1] + b_2 x[n-2] + ...$
 - $y[n]$ is the output signal at time $n$
 - $x[n]$ is the input signal
 - $b_i$ are the filter coefficients or impulse response
 - $n-1$, $n-2$, ... are time $n$ delayed by 1, 2, ... cycles
 
### Specification

Build a 4-element FIR filter where the four filter coefficients are parameters. A module skeleton and basic tests are provided for you.
Note that both the input and output are 8-bit single elements, you will need to save necessary state (like delayed signal values) using constructs like shift registers.

In [10]:
// Run this boilerplate for the necessary imports

import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::firrtl:1.0-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import $ivy.$                                                 

import $ivy.$                                                          

import $ivy.$                                                

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

In [12]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  // YOUR CODE HERE
    
  // Reference solution
  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + x_n3 * b3.U(8.W)
}

defined class My4ElementFir

In [8]:
// Simple sanity check: a element with all zero coefficients should always produce zero
Driver(() => new My4ElementFir(0, 0, 0, 0)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 5)
    expect(c.io.out, 0)
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 0)
  }
}

[info] [0.000] Elaborating design...
[info] [0.044] Done elaborating.
Total FIRRTL Compile Time: 46.2 ms
Total FIRRTL Compile Time: 24.2 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1507312095895
test cmd6WrapperHelperMy4ElementFir Success: 4 tests passed in 8 cycles taking 0.011465 seconds
[info] [0.008] RAN 3 CYCLES PASSED


res7: Boolean = true

In [9]:
// Simple 4-point moving average
Driver(() => new My4ElementFir(1, 1, 1, 1)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1, 0, 0, 0
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 5)  // 4, 1, 0, 0
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 8)  // 3, 4, 1, 0
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 10)  // 2, 3, 4, 1
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 16)  // 7, 2, 3, 4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 12)  // 0, 7, 2, 3
  }
}

[info] [0.000] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 17.9 ms
Total FIRRTL Compile Time: 38.0 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1507312157369
test cmd6WrapperHelperMy4ElementFir Success: 6 tests passed in 10 cycles taking 0.022314 seconds
[info] [0.016] RAN 5 CYCLES PASSED


res8: Boolean = true

In [11]:
// Nonsymmetric filter
Driver(() => new My4ElementFir(1, 2, 3, 4)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 1)
    expect(c.io.out, 1)  // 1*1, 0*2, 0*3, 0*4
    step(1)
    poke(c.io.in, 4)
    expect(c.io.out, 6)  // 4*1, 1*2, 0*3, 0*4
    step(1)
    poke(c.io.in, 3)
    expect(c.io.out, 14)  // 3*1, 4*2, 1*3, 0*4
    step(1)
    poke(c.io.in, 2)
    expect(c.io.out, 24)  // 2*1, 3*2, 4*3, 1*4
    step(1)
    poke(c.io.in, 7)
    expect(c.io.out, 36)  // 7*1, 2*2, 3*3, 4*4
    step(1)
    poke(c.io.in, 0)
    expect(c.io.out, 32)  // 0*1, 7*2, 2*3, 3*4
  }
}

[info] [0.000] Elaborating design...
[info] [0.000] Done elaborating.
Total FIRRTL Compile Time: 13.8 ms
Total FIRRTL Compile Time: 11.2 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1507312515248
test cmd6WrapperHelperMy4ElementFir Success: 6 tests passed in 10 cycles taking 0.037760 seconds
[info] [0.016] RAN 5 CYCLES PASSED


res10: Boolean = true